In [1]:
from dataclasses import dataclass

from pydantic_ai import Agent, RunContext, ModelRetry
from pydantic_ai.models import ModelResponse
from pydantic_ai.messages import ToolCallPart

from pydanticai_examples.config import get_qdrant_config, get_openai_config, get_todoist_config
from pydanticai_examples.vector_store import VectorStore
from pydanticai_examples.todoist import TodoistClient
from dotenv import load_dotenv
from pydanticai_examples.utils import get_root


In [2]:
load_dotenv(get_root() / ".env")

True

In [3]:
@dataclass
class RecipeAgentDeps:
    """Class to represent the dependencies for the Recipe Agent"""
    vector_store: VectorStore
    todoist_client: TodoistClient

In [4]:
@dataclass
class ChosenRecipe:
    """Class to represent a chosen recipe"""
    name: str
    description: str
    ingredients: list[str]
    instructions: str

    def __str__(self):
        return f"{self.name}: {self.description}\nIngredients: {', '.join(self.ingredients)}\nInstructions: {self.instructions}"

In [5]:
cooking_agent = Agent[RecipeAgentDeps, ChosenRecipe](
    model='openai:gpt-4-turbo',
    deps_type=RecipeAgentDeps,
    result_type=ChosenRecipe,
    system_prompt="""You are a professional dietitian. For a given list of ingredients that a user has in their stash, you should suggest a recipe that uses those ingredients.
You should only use recipes that are contained in the vector database.

If the list of ingredients provided by the user is missing some of the ingredients from the chosen recipe, create a Todoist task to buy a missing ingredients.
The task title should be "Buy missing ingredients for {recipe_name}".
The task details should include the list of missing ingredients.
    """
)

In [6]:

@cooking_agent.tool
async def search_for_recipes(ctx: RunContext[RecipeAgentDeps], ingredients: list[str]) -> list[dict]:
    """Search the vector store for cooking recipes
    including the ingredients provided.

    Args:
        ingredients (list[str]): The list of ingredients the resulting
            recipes should include.

    Returns:
        list[str]: List of recipes found for a given query.
    """
    # Embed the query
    ingredients_str = ", ".join(ingredients)
    return await ctx.deps.vector_store.search(ingredients_str)

In [7]:
@cooking_agent.tool
async def add_task_to_todoist(ctx: RunContext[RecipeAgentDeps], task_title: str, task_details: str) -> list[dict]:
    """Add a task to Todoist todo app.

    Args:
        task_title (str): The title of the task.
        task_details (str): The details of the task.

    Returns:
        list[dict]: The created task.
    """
    return await ctx.deps.todoist_client.add_task(task_title, task_details)

In [8]:
@cooking_agent.result_validator
async def check_if_task_added(ctx: RunContext[RecipeAgentDeps], result: ChosenRecipe) -> ChosenRecipe:
    """Validator to check if a task was added to Todoist.

    Args:
        ctx (RunContext[RecipeAgentDeps]): The context of the run.
        result (ChosenRecipe): The result of the agent.

    Returns:
        ChosenRecipe: The result of the agent.

    Raises:
        ModelRetry: If the agent did not create a task to buy missing ingredients.
    """
    todoist_used = False
    for msg in ctx.messages:
        if isinstance(msg, ModelResponse):
            for part in msg.parts:
                if isinstance(part, ToolCallPart):
                    if part.tool_name == "add_task_to_todoist":
                        todoist_used = True
                        break
    if not todoist_used:
        raise ModelRetry("The agent did not create a task to buy missing ingredients.")
    return result


In [9]:
# Instantiate the vector store
qdrant_config = get_qdrant_config()
openai_config = get_openai_config()
vector_store = VectorStore(
    url=qdrant_config.qdrant_url,
    port=qdrant_config.qdrant_port,
    collection_name=qdrant_config.qdrant_index_name,
    openai_api_key=openai_config.openai_api_key,
)

# Instantiate the Todoist client
todoist_config = get_todoist_config()
todoist_client = TodoistClient(
    api_key=todoist_config.todoist_api_key,
    project=todoist_config.todoist_project,
)

# Gather dependencies into a single object
deps = RecipeAgentDeps(
    vector_store=vector_store,
    todoist_client=todoist_client,
)

In [10]:
result = await cooking_agent.run("I have only milk and wheat", deps=deps)

In [11]:
result.data

In [12]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a professional dietitian. For a given list of ingredients that a user has in their stash, you should suggest a recipe that uses those ingredients.\nYou should only use recipes that are contained in the vector database.\n\nIf the list of ingredients provided by the user is missing some of the ingredients from the chosen recipe, create a Todoist task to buy a missing ingredients.\nThe task title should be "Buy missing ingredients for {recipe_name}".\nThe task details should include the list of missing ingredients.\n    ', timestamp=datetime.datetime(2025, 4, 2, 18, 52, 56, 201034, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='I have only milk and wheat', timestamp=datetime.datetime(2025, 4, 2, 18, 52, 56, 201036, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='search_for_recipes', args='{"ingredients":["milk

In [19]:
result = await cooking_agent.run("Why didn't you create a task to buy the missing ingredients?",
                                 deps=deps, message_history=result.new_messages())

In [20]:
result.new_messages()

[ModelRequest(parts=[UserPromptPart(content="Why didn't you create a task to buy the missing ingredients?", timestamp=datetime.datetime(2025, 3, 30, 14, 25, 45, 321186, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='add_task_to_todoist', args='{"task_title":"Buy missing ingredients for French Toast with Cinnamon","task_details":"Eggs, Sugar, Cinnamon, Butter, Bread"}', tool_call_id='call_SoxUpr3Pi0GxB06oVZYgLmca', part_kind='tool-call')], model_name='gpt-4-turbo-2024-04-09', timestamp=datetime.datetime(2025, 3, 30, 14, 25, 45, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='add_task_to_todoist', content=Task(assignee_id=None, assigner_id=None, comment_count=0, is_completed=False, content='Buy missing ingredients for French Toast with Cinnamon', created_at='2025-03-30T14:25:49.058141Z', creator_id='18079813', description='Eggs, Sugar, Cinnamon, Butter, Bread', due=No